In [ ]:
import datetime
import math
import pathlib

import folium
import geopandas
import joblib
import pygeoops
import shapely
import urllib.parse
import xyzservices

In [ ]:
CACHE_DIRECTORY = pathlib.Path() / ".cache"
CACHE_TIME = datetime.timedelta(weeks=3)

CACHE_DIRECTORY.mkdir(exist_ok=True)
memory = joblib.Memory(CACHE_DIRECTORY, verbose=0)
memory.reduce_size(age_limit=CACHE_TIME)

In [ ]:
MINDESTBREITE = 1.5  # meter
MINDESTFLAECHE = 20  # quadratmeter

CRS = "EPSG:31256"

WFS_BASE_URL = (
    "https://data.wien.gv.at/daten/geo"
    "?service=WFS"
    "&version=2.0.0"
    "&request=GetFeature"
    f"&srsName={CRS}"
    "&typeNames={:s}"
)

PARKFLAECHEN_AUF_GEHSTEIG_WFS_URL = (
    WFS_BASE_URL.format("ogdwien:SISBELAGOGD")
    + "&"
    + urllib.parse.urlencode({"cql_filter": "TYPE_TXT='Parkfläche Gehsteig'"})
)
GEHSTEIG_WFS_URL = (
    WFS_BASE_URL.format("ogdwien:SISBELAGOGD")
    + "&"
    + urllib.parse.urlencode({"cql_filter": "TYPE_TXT='Gehsteig'"})
)
ORTHOFOTO_TILE_URL = (
    "https://mapsneu.wien.gv.at/wmts/lb/farbe/google3857/{z}/{y}/{x}.jpeg"
)
xyzservices.providers.WienOGD = xyzservices.Bunch(
    Orthofoto=xyzservices.TileProvider(
        name="Orthofoto Wien",
        url=ORTHOFOTO_TILE_URL,
        attribution="Datenquelle: Stadt Wien – data.wien.gv.at",
    )
)

In [ ]:
MAX_DISTANCE = 0.5

In [ ]:
GEHSTEIGE_FILL_COLOUR = "#83ba7e"
GEHSTEIGE_STROKE_COLOUR = "#83ba7e"
PARKPLAETZE_FILL_COLOUR = "#73a0dc"
PARKPLAETZE_STROKE_COLOUR = "#73a0dc"

In [ ]:
@memory.cache
def cached_geopandas_read_file(*args, **kwargs):
    return geopandas.read_file(*args, **kwargs)

In [ ]:
parkflaechen = cached_geopandas_read_file(PARKFLAECHEN_AUF_GEHSTEIG_WFS_URL)
parkflaechen.crs = CRS
parkflaechen = parkflaechen.set_index("gml_id")
parkflaechen

In [ ]:
gehsteig = cached_geopandas_read_file(GEHSTEIG_WFS_URL)
gehsteig.crs = CRS
gehsteig = gehsteig.set_index("gml_id")
gehsteig

In [ ]:
gehsteig_neben_parkflaeche = gehsteig.sjoin_nearest(
    parkflaechen,
    max_distance=MAX_DISTANCE,
    lsuffix="gehsteig",
    rsuffix="parkflaeche",
    # distance_col="Abstand Gehsteig/Parkfläche",
)[["geometry"]].dissolve().explode().reset_index(drop=True)
gehsteig_neben_parkflaeche

In [ ]:
gehsteig_neben_parkflaeche["centerline"] = pygeoops.centerline(gehsteig_neben_parkflaeche.geometry)

In [ ]:
def durchschnittsbreite(polygon):
    # Die durchschnittliche Breite eines Polygons berechnen/schätzen
    #
    # Der Algorithmus ist an den (hier)[https://web.archive.org/web/20230329134916/https://geoobserver.wordpress.com/2023/03/27/qgis-tipp-flussbreiten-ermitteln-aber-wie/]
    # besprochenen Ansatz zur Flußbreitenbeschriftung
    # angelehnt.
    #
    # Wir konstruieren zunächst Linestrings die an 10 gleichmäßig verteilten
    # Punkten normal auf die Centerline des Polygons stehen, clippen diese Linien
    # dann mit dem Polygon, und berechnen deren Länge (und schließlich den
    # Durchschnitt dieser Längen
    
    MAX_LAENGE_NORMALE = 30  # wenn gehsteige breiter als 30 meter sind sind sie uns eh wurscht
    
    zwoelf_punkte = [
        shapely.line_interpolate_point(
            pygeoops.centerline(polygon),
            distance=(i * 1.0/11.0),
            normalized=True,
        ) for i in range(12)
    ]

    normale = []
    for (previous_point, point, next_point) in zip(
        zwoelf_punkte[:-2], zwoelf_punkte[1:-1], zwoelf_punkte[2:]
    ):
        dx = next_point.x - previous_point.x
        dy = next_point.y - previous_point.y
        laenge = math.sqrt(dx**2 + dy**2)
        dx /= laenge
        dy /= laenge
        normale.append(
            shapely.intersection(
                shapely.LineString(
                    [
                        [point.x - (dx * laenge), point.y + (dy * laenge)],
                        [point.x + (dx * laenge), point.y - (dy * laenge)],
                    ],
                ),
                polygon
            )
        )

    normale = [shapely.intersection(n, polygon) for n in normale]
    laengen = [shapely.length(n) for n in normale]
    average_length = sum(laengen) / len(laengen)
    
    return average_length    

In [ ]:
gehsteig_neben_parkflaeche["breite"] = gehsteig_neben_parkflaeche.geometry.apply(durchschnittsbreite)

In [ ]:
gehsteig_neben_parkflaeche["breite"].describe()

In [ ]:
gehsteig_neben_parkflaeche = gehsteig_neben_parkflaeche[(gehsteig_neben_parkflaeche.breite < MINDESTBREITE) & (gehsteig_neben_parkflaeche.area > MINDESTFLAECHE)][["geometry", "breite"]]
gehsteig_neben_parkflaeche

In [ ]:
gehsteig_neben_parkflaeche.to_file("/tmp/gehsteig_neben_parkflaeche.gpkg")
parkflaechen.to_file("/tmp/parkflaechen.gpkg")

In [ ]:
tile_layer = folium.raster_layers.TileLayer(xyzservices.providers.WienOGD.Orthofoto, opacity=0.1)

uebersichtskarte = gehsteig_neben_parkflaeche.explore(
    style_kwds={
        "fillColor": GEHSTEIGE_FILL_COLOUR,
        "fillOpacity": 0.9,
        "color": GEHSTEIGE_STROKE_COLOUR,
        "weight": 0.1,
    },
    map_kwds={
        "center": {"lat": 48.2515, "lng": 16.4549},
    },
    zoom_start=18,
    max_zoom=32,
    tiles=tile_layer
    #tiles=xyzservices.providers.WienOGD.Orthofoto,
)
uebersichtskarte = parkflaechen.explore(
    m=uebersichtskarte,
    style_kwds={
        "fillColor": PARKPLAETZE_FILL_COLOUR,
        "fillOpacity": 0.9,
        "color": PARKPLAETZE_STROKE_COLOUR,
        "weight": 0.1,
    },
)
uebersichtskarte